In [ ]:
# default_exp models.deepim

# DeepIM
> Implementation of DeepIM model in PyTorch.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.nb_imports import *
from fastcore.test import *

In [ ]:
#export
import torch
from torch import nn

from recohut.models.layers.embedding import EmbeddingLayer
from recohut.models.layers.common import MLP_Layer
from recohut.models.layers.interaction import InteractionMachine

from recohut.models.bases.ctr import CTRModel

In [ ]:
#export
class DeepIM(CTRModel):
    def __init__(self, 
                 feature_map, 
                 model_id="DeepIM",
                 task="binary_classification",
                 learning_rate=1e-3, 
                 embedding_initializer="torch.nn.init.normal_(std=1e-4)",
                 embedding_dim=10, 
                 im_order=2, 
                 im_batch_norm=False,
                 hidden_units=[64, 64, 64], 
                 hidden_activations="ReLU", 
                 net_dropout=0, 
                 batch_norm=False, 
                 **kwargs):
        super(DeepIM, self).__init__(feature_map, 
                                           model_id=model_id,
                                           **kwargs)
        self.embedding_layer = EmbeddingLayer(feature_map, embedding_dim)
        self.im_layer = InteractionMachine(embedding_dim, im_order, im_batch_norm)
        self.dnn = MLP_Layer(input_dim=embedding_dim * feature_map.num_fields,
                             output_dim=1,
                             hidden_units=hidden_units,
                             hidden_activations=hidden_activations,
                             output_activation=None,
                             dropout_rates=net_dropout,
                             batch_norm=batch_norm) \
                   if hidden_units is not None else None
        self.output_activation = self.get_final_activation(task)
        self.init_weights(embedding_initializer=embedding_initializer)

    def forward(self, inputs):
        feature_emb = self.embedding_layer(inputs)
        y_pred = self.im_layer(feature_emb)
        if self.dnn is not None:
            y_pred += self.dnn(feature_emb.flatten(start_dim=1))
        if self.output_activation is not None:
            y_pred = self.output_activation(y_pred)
        return y_pred

Example

In [ ]:
params = {'model_id': 'DeepIM',
              'data_dir': '/content/data',
              'model_root': './checkpoints/',
              'learning_rate': 1e-3,
              'optimizer': 'adamw',
              'task': 'binary_classification',
              'loss': 'binary_crossentropy',
              'metrics': ['logloss', 'AUC'],
              'embedding_dim': 10,
              'im_order': 2,
              'im_batch_norm': False,
              'hidden_units': [300, 300, 300],
              'hidden_activations': 'relu',
              'net_regularizer': 0,
              'embedding_regularizer': 0,
              'net_batch_norm': False,
              'net_dropout': 0,
              'batch_size': 64,
              'epochs': 3,
              'shuffle': True,
              'seed': 2019,
              'use_hdf5': True,
              'workers': 1,
              'verbose': 0}

In [ ]:
model = DeepIM(ds.dataset.feature_map, **params)

In [ ]:
pl_trainer(model, ds, max_epochs=5)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name              | Type               | Params
---------------------------------------------------------
0 | embedding_layer   | EmbeddingLayer     | 4.8 K 
1 | im_layer          | InteractionMachine | 21    
2 | dnn               | MLP_Layer          | 223 K 
3 | output_activation | Sigmoid            | 0     
---------------------------------------------------------
227 K     Trainable params
0         Non-trainable params
227 K     Total params
0.912     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'Test Metrics': {'AUC': tensor(1.), 'logloss': tensor(0.1478)}}
--------------------------------------------------------------------------------


[{'Test Metrics': {'AUC': tensor(1.), 'logloss': tensor(0.1478)}}]